https://www.kaggle.com/datasets/bohnacker/country-longitude-latitude?resource=download


In [29]:
import json

import numpy as np
import pandas as pd

In [30]:
MIN_BOOKS_THRESHOLD = 0

In [31]:
file_path = './data/hotel_bookings.csv'
bookings_df = pd.read_csv(file_path)

print(bookings_df.columns)

bookings_df.head()

Index(['Unnamed: 0', 'hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date', 'dia', 'tipo'],
      dtype='object')


,Unnamed: 0,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,dia,tipo
0,3,Resort Hotel,0,7,2015,July,27,1,0,1,...,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02,2015-07-01,work
1,4,Resort Hotel,0,13,2015,July,27,1,0,1,...,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02,2015-07-01,work
2,5,Resort Hotel,0,14,2015,July,27,1,0,2,...,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03,2015-07-01,work
3,6,Resort Hotel,0,14,2015,July,27,1,0,2,...,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03,2015-07-01,work
4,7,Resort Hotel,0,0,2015,July,27,1,0,2,...,NaN,0,Transient,107.0,0,0,Check-Out,2015-07-03,2015-07-01,work


In [32]:
grouped_countries = bookings_df.groupby(['country']).agg(
    total_books = ('is_canceled', 'count')
).reset_index()

grouped_countries = grouped_countries[grouped_countries['total_books'] >= MIN_BOOKS_THRESHOLD]

relevant_countries = grouped_countries['country']
num_books_before_clean = bookings_df.shape[0]

bookings_df = bookings_df[bookings_df['country'].isin(relevant_countries)]

num_books_after_clean = bookings_df.shape[0]

print(f'A total {num_books_before_clean-num_books_after_clean} of books have been cleaned')

A total 467 of books have been cleaned


In [33]:
file_path = './data/countries.csv'
countries_df = pd.read_csv(file_path)


countries_df.head()

repeated = countries_df['ISO-ALPHA-3'].value_counts()[countries_df['ISO-ALPHA-3'].value_counts() > 1].index

for country in repeated.to_list():
    if country in bookings_df['country'].unique():
        print(countries_df[countries_df['ISO-ALPHA-3'] == country][['Country', 'ISO-ALPHA-3']])

unused_countries = ["Russian Federation", "China, People's Republic of", "United States", 
                    "United Kingdom of Great Britain and Northern Ireland", "West Germany"]

countries_df = countries_df[~countries_df['Country'].isin(unused_countries)]

                        Country ISO-ALPHA-3
154  Korea, Republic of (South)         KOR
155                 South Korea         KOR
156          Korea, Republic of         KOR
                        Country ISO-ALPHA-3
53                        China         CHN
54  China, People's Republic of         CHN
                Country ISO-ALPHA-3
233  Russian Federation         RUS
234              Russia         RUS
        Country ISO-ALPHA-3
30      Bahamas         BHS
31  The Bahamas         BHS
                   Country ISO-ALPHA-3
253  São Tomé and Príncipe         STP
254  Sao Tome and Principe         STP
               Country ISO-ALPHA-3
182    North Macedonia         MKD
183  Macedonia [FYROM]         MKD
           Country ISO-ALPHA-3
74  Czech Republic         CZE
75         Czechia         CZE
                             Country ISO-ALPHA-3
37                           Bolivia         BOL
38  Bolivia (Plurinational State of)         BOL
                          Country ISO-

In [34]:
country_to_iso = {}
iso_to_country = {}


for country in bookings_df['country'].unique():
    iso = countries_df[countries_df['ISO-ALPHA-3'] == country]['Country'].values
    if len(iso) == 0:
        print(country)
        continue
    country_to_iso[iso[0]] = country
    iso_to_country[country] = iso[0]

iso_to_country['CN'] = 'China'
print(country_to_iso)
print(iso_to_country)

CN
TMP
UMI
{'United Kingdom': 'GBR', 'Portugal': 'PRT', 'United States of America': 'USA', 'Spain': 'ESP', 'Ireland': 'IRL', 'France': 'FRA', 'Romania': 'ROU', 'Norway': 'NOR', 'Oman': 'OMN', 'Argentina': 'ARG', 'Poland': 'POL', 'Germany': 'DEU', 'Belgium': 'BEL', 'Switzerland': 'CHE', 'Greece': 'GRC', 'Italy': 'ITA', 'Netherlands': 'NLD', 'Denmark': 'DNK', 'Russia': 'RUS', 'Sweden': 'SWE', 'Australia': 'AUS', 'Estonia': 'EST', 'Czech Republic': 'CZE', 'Brazil': 'BRA', 'Finland': 'FIN', 'Mozambique': 'MOZ', 'Botswana': 'BWA', 'Luxembourg': 'LUX', 'Slovenia': 'SVN', 'Albania': 'ALB', 'India': 'IND', 'China': 'CHN', 'Mexico': 'MEX', 'Morocco': 'MAR', 'Ukraine': 'UKR', 'San Marino': 'SMR', 'Latvia': 'LVA', 'Puerto Rico': 'PRI', 'Serbia': 'SRB', 'Chile': 'CHL', 'Austria': 'AUT', 'Belarus': 'BLR', 'Lithuania': 'LTU', 'Turkey': 'TUR', 'South Africa': 'ZAF', 'Israel': 'ISR', 'Cayman Islands': 'CYM', 'Zambia': 'ZMB', 'Cape Verde': 'CPV', 'Zimbabwe': 'ZWE', 'Algeria': 'DZA', 'Korea, Republic of

In [35]:
bookings_df.head()

,Unnamed: 0,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,dia,tipo
0,3,Resort Hotel,0,7,2015,July,27,1,0,1,...,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02,2015-07-01,work
1,4,Resort Hotel,0,13,2015,July,27,1,0,1,...,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02,2015-07-01,work
2,5,Resort Hotel,0,14,2015,July,27,1,0,2,...,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03,2015-07-01,work
3,6,Resort Hotel,0,14,2015,July,27,1,0,2,...,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03,2015-07-01,work
4,7,Resort Hotel,0,0,2015,July,27,1,0,2,...,NaN,0,Transient,107.0,0,0,Check-Out,2015-07-03,2015-07-01,work


In [36]:
charts_info_dict = {}

In [37]:
num_canceled = sum(bookings_df['is_canceled'] == 1)
num_confirmed = sum(bookings_df['is_canceled'] == 0)
cancel_rate = num_canceled/(num_canceled+num_confirmed) * 100
print(f'There is a total of {num_canceled} of canceled reservations')
print(f'There is a total of {num_confirmed} of confirmed reservations')
print(f'The cancel rate is: {cancel_rate}')

charts_info_dict['chart1'] = {'confirmed': num_confirmed, 'canceled': num_canceled}

There is a total of 43948 of canceled reservations
There is a total of 72976 of confirmed reservations
The cancel rate is: 37.586808525195856


# Chart 2: Mapa mundial

Per cada país cal tenir:
<ul>
  <li> localització
  <li> total reserves (confirmades/cancelades)
  <li> total reserves per tipus de grup (familia, parella, sol)
  <li> total reserves per lloc de reserva (online, no-online)
  <li> total reserves per antel·lació en mesos( 1, 2, 3, 4, 5, 6, 7+)
</ul>

In [38]:
cancel_summary = bookings_df.groupby(['country']).agg(
    total_books = ('is_canceled', 'count'),
    cancelations = ('is_canceled', 'sum')
).reset_index()

cancel_summary = cancel_summary.loc[cancel_summary['total_books'] >= MIN_BOOKS_THRESHOLD]

cancel_summary['confirmations'] = cancel_summary['total_books'] - cancel_summary['cancelations']
cancel_summary['cancelations_rate'] = cancel_summary['cancelations'] / cancel_summary['total_books'] * 100

cancel_summary = cancel_summary.sort_values(by='cancelations_rate', ascending=False)

print(cancel_summary)


    country  total_books  cancelations  confirmations  cancelations_rate
88      KHM            2             2              0              100.0
16      BEN            3             3              0              100.0
61      GGY            3             3              0              100.0
64      GLP            2             2              0              100.0
70      HND            1             1              0              100.0
..      ...          ...           ...            ...                ...
114     MRT            1             0              1                0.0
111     MMR            1             0              1                0.0
105     MDG            1             0              1                0.0
26      BRB            4             0              4                0.0
0       ABW            2             0              2                0.0

[177 rows x 5 columns]


In [39]:
charts_info_dict['chart2'] = {}

for _, country_data in cancel_summary.iterrows():
    country = country_data['country']
    charts_info_dict['chart2'][iso_to_country[country]] = {
        'total_books': country_data['total_books'],
        'cancelations': country_data['cancelations'],
        'confirmations': country_data['confirmations'],
        'cancelations_rate': country_data['cancelations_rate']
    }

KeyError: 'UMI'

In [ ]:
conditions = [
    (bookings_df['lead_time'] > 180),
    (bookings_df['lead_time'] <= 180) & (bookings_df['lead_time'] > 150),
    (bookings_df['lead_time'] <= 150) & (bookings_df['lead_time'] > 120),
    (bookings_df['lead_time'] <= 120) & (bookings_df['lead_time'] > 90),
    (bookings_df['lead_time'] <= 90) & (bookings_df['lead_time'] > 60),
    (bookings_df['lead_time'] <= 60) & (bookings_df['lead_time'] > 30),
    (bookings_df['lead_time'] <= 30)
]

values = [7, 6, 5, 4, 3, 2, 1]

bookings_df['lead_months'] = np.select(conditions, values)
bookings_df['lead_months'].value_counts()

lead_months
1    34243
7    23731
2    15756
3    11813
4     9804
5     7704
6     7528
Name: count, dtype: int64

In [ ]:
bookings_df['distribution_channel'].value_counts()

distribution_channel
TA/TO        91361
Direct       12938
Corporate     6104
GDS            175
Undefined        1
Name: count, dtype: int64

In [ ]:
output_json_path = '../hotel-booking/public/data/hotel_bookings.json'
with open(output_json_path, 'w', encoding='utf-8') as file:
    json.dump(charts_info_dict, file, ensure_ascii=False, indent=4)